In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import sys
import msgpack as mp
import msgpack_numpy as mpn
sys.path.append(os.path.join(os.getcwd(), ".."))
from support.generate_ar_data import *
from support.pd_support import *
from support.calculations_support import *
from support.mecanum_calculations import *
from support.ar_calculations import *
import pandas as pd
from scipy.signal import savgol_filter
from scipy.interpolate import interp1d

In [2]:
_parent_folder = "multi_cam_nov_14"
_base_pth = os.path.dirname(os.getcwd())
_base_pth = os.path.join(_base_pth, "recording_programs", "test_data", _parent_folder)
_base_pth

'c:\\Users\\Sujith\\Documents\\Projects\\aruco\\recording_programs\\test_data\\multi_cam_nov_14'

In [3]:
""" read excel sheet"""
folder_df = pd.read_excel(os.path.join(_base_pth, "00_analysis", "calibration_files.xlsx"))
folder_df

,folder_name,calibration_folder,flip
0,sk16_15_forward_fast,calibration_15fps_sk,False
1,sk16_15_forward_fast_2,calibration_15fps_sk,False
2,sk16_15_forward_medium,calibration_15fps_sk,False
3,sk16_15_forward_medium_2,calibration_15fps_sk,False
4,sk16_15_forward_slow,calibration_15fps_sk,False
...,...,...,...
84,sk16_60_sideways_fast_2,calibration_60fps_quad_sk_3,False
85,sk16_60_sideways_medium,calibration_60fps_quad_sk_3,False
86,sk16_60_sideways_medium_2,calibration_60fps_quad_sk_3,False
87,sk16_60_sideways_slow,calibration_60fps_quad_sk_3,False


In [4]:

def transform_coordinates_1(df, rotmat, org):
    df1 = df.copy( deep=True)
    for i in range(len(df)):
        val = df[["x", "y", "z"]].loc[i].values
        val = np.reshape(val, (3,1))
        _temp = val - org
        translation_correction = np.array([0.045, -0.05, 0.045]).reshape(3, 1)
        _val = rotmat.T @ _temp
        _val = _val - translation_correction
        
        df1["x"].replace({df1["x"].loc[i]: _val[0]}, inplace=True)
        df1["y"].replace({df1["y"].loc[i]: _val[1]}, inplace=True)
        df1["z"].replace({df1["z"].loc[i]: _val[2]}, inplace=True)

    return df1

In [5]:
mean_err_df = pd.DataFrame(columns=["folder_name", "err_x", "err_y", "err_z", "max_err_x", "max_err_y", "max_err_z", "std_err_x", "std_err_y", "std_err_z"])

In [9]:
for i in range(len(folder_df)):
    folder_no = i
    #open the calibration files
    _pth = folder_df.folder_name[folder_no]
    _pth = os.path.join(_base_pth, _pth)

    #read the calibration file
    _webcam_calib_folder = folder_df.calibration_folder[folder_no]
    _webcam_calib_folder = os.path.join(_base_pth, _webcam_calib_folder)
    _webcam_calib_pth = os.path.join(_base_pth, _webcam_calib_folder, "webcam_calibration.msgpack")


    with open(_webcam_calib_pth, "rb") as f:
        webcam_calib = mp.Unpacker(f, object_hook=mpn.decode)
        _temp = next(webcam_calib)
        _webcam_cam_mat = _temp[0]
        _webcam_dist = _temp[1]
    _webcam_video_pth = os.path.join(_pth, "webcam_color.msgpack")
    _webcam_timestamp_pth = os.path.join(_pth, "webcam_timestamp.msgpack")


    with open(os.path.join(_webcam_calib_folder, "webcam_rotmat.msgpack"), "rb") as f:
        webcam_rotmat = mp.Unpacker(f, object_hook=mpn.decode)
        _webcam_rot = next(webcam_rotmat)
        _webcam_org = next(webcam_rotmat)

    
    _w_df = get_ar_pose_data(_webcam_video_pth, _webcam_cam_mat, _webcam_dist, is_color=False, single_file=True, flip_frame=False)

    _w_df = add_time_from_file(_w_df, _webcam_timestamp_pth)
    w_df = transform_coordinates_1(_w_df, _webcam_rot, _webcam_org)

    mc_df_pth = os.path.join(_base_pth, folder_df.folder_name[folder_no].split("_")[0], folder_df.folder_name[folder_no]+ ".csv")
    _m_df, st_time = read_df_csv(mc_df_pth)
    _m_df = add_datetime_col(_m_df, st_time, _name="seconds")


    _m_df["x"] = _m_df["lb_x"] + _m_df["rb_x"] + _m_df["lt_x"] + _m_df["rt_x"]
    _m_df["y"] = _m_df["lb_y"] + _m_df["rb_y"] + _m_df["lt_y"] + _m_df["rt_y"]
    _m_df["z"] = _m_df["lb_z"] + _m_df["rb_z"] + _m_df["lt_z"] + _m_df["rt_z"]

    _m_df["x"] = _m_df["x"] / 4
    _m_df["y"] = _m_df["y"] / 4
    _m_df["z"] = _m_df["z"] / 4

    w_df, _m_df = trunkate_dfs(w_df, _m_df)

    w_df = interpolate_target_df(w_df, _m_df)

    error_df = pd.DataFrame()
    error_df["time"] = _m_df["time"]
    error_df["x"] = _m_df["x"] - w_df["x"]
    error_df["y"] = _m_df["y"] - w_df["y"]
    error_df["z"] = _m_df["z"] - w_df["z"]

    mean_err_df = mean_err_df.append({"folder_name": folder_df.folder_name[folder_no], "err_x": error_df["x"].mean(), "err_y": error_df["y"].mean(), "err_z": error_df["z"].mean(), 
                                      "max_err_x": error_df["x"].max(), "max_err_y": error_df["y"].max(), "max_err_z": error_df["z"].max(), 
                                      "std_err_x": error_df["x"].std(), "std_err_y": error_df["y"].std(), "std_err_z": error_df["z"].std()}, ignore_index=True)
    
    print("processed", i)

returning dataframe
processed 0
returning dataframe
processed 1
returning dataframe
processed 2
returning dataframe
processed 3
returning dataframe
processed 4
returning dataframe
processed 5
returning dataframe
processed 6
returning dataframe
processed 7
returning dataframe
processed 8
returning dataframe
processed 9
returning dataframe
processed 10
returning dataframe
processed 11
returning dataframe
processed 12
returning dataframe
processed 13
returning dataframe
processed 14
returning dataframe
processed 15
returning dataframe
processed 16
returning dataframe
processed 17
returning dataframe
processed 18
returning dataframe
processed 19
returning dataframe
processed 20
returning dataframe
processed 21
returning dataframe
processed 22
returning dataframe
processed 23
returning dataframe
processed 24
returning dataframe
processed 25
returning dataframe
processed 26
returning dataframe
processed 27
returning dataframe
processed 28
returning dataframe
processed 29
returning dataframe


KeyboardInterrupt: 

In [10]:
mean_err_df

,folder_name,err_x,err_y,err_z,max_err_x,max_err_y,max_err_z,std_err_x,std_err_y,std_err_z
0,sk16_15_forward_fast,0.000526,-0.012339,0.015757,NaN,NaN,NaN,NaN,NaN,NaN
1,sk16_15_forward_fast,0.000526,-0.012339,0.015757,NaN,NaN,NaN,NaN,NaN,NaN
2,sk16_15_forward_fast_2,-0.000540,-0.012633,0.013277,NaN,NaN,NaN,NaN,NaN,NaN
3,sk16_15_forward_medium,-0.000363,-0.012625,0.012512,NaN,NaN,NaN,NaN,NaN,NaN
4,sk16_15_forward_medium_2,-0.000078,-0.012516,0.013455,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
121,sk16_30_sideways_medium_2,-0.018810,-0.003882,0.059428,0.382815,0.077472,0.469376,0.185003,0.004067,0.021749
122,sk16_30_sideways_slow,0.029295,-0.005352,0.053802,0.406853,0.004941,0.114109,0.193500,0.003393,0.019448
123,sk16_30_sideways_slow_2,-0.013868,-0.004260,0.058506,0.382750,0.087802,0.536957,0.201159,0.010312,0.053989
124,sk16_60_forward_fast,-0.005685,-0.006047,0.037282,0.013031,0.005089,0.094854,0.007545,0.002066,0.018875


In [ ]:
plt.plot(_m_df["time"], _m_df["z"], label="mc")
# plt.plot(_w_df["time"], _w_df["z"].rolling(13).median()*-1  +1.5, label="webcam")
plt.plot(w_df["time"], savgol_filter(w_df["z"].values, 51, 1, mode='nearest') + 0.045, label="webcam")
plt.legend()

In [ ]:
# find error between x , y, z and yaw, pitch, roll
error_df = pd.DataFrame()
error_df["time"] = _m_df["time"]
error_df["x"] = _m_df["x"] - w_df["x"]
error_df["y"] = _m_df["y"] - w_df["y"]
error_df["z"] = _m_df["z"] - w_df["z"]

In [ ]:
print(error_df.x.mean())
print(error_df.y.mean())
print(error_df.z.mean())